In [6]:
# Ingest the index
from llama_index import StorageContext, load_index_from_storage
import os

# PLEASE MAKE SURE YOU SET THE OPENAI KEY otherwise nothing will work.

# import langchain 
#langchain.debug = True

In [7]:
project_path="./bioe80"
storage_path=project_path+"/storage"

if os.path.isfile(project_path+"/storage/index_store.json") :
    print("Found index - loading - this may take a few seconds")
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=storage_path)
    # load index
    index = load_index_from_storage(storage_context)
else :
	print("please generate an index first")

print(index)

Found index - loading - this may take a few seconds


In [8]:
from langchain.agents import Tool

tools = [
    Tool(
        name="BITS",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about your class. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [9]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-3.5-turbo || gpt-4
# llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-4") # gpt-3.5-turbo || gpt-4
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory, verbose=True)

In [10]:
agent_chain.run(input="Hi, I am Drew")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Drew! How can I assist you today?

> Finished chain.


'Hello Drew! How can I assist you today?'

In [11]:
cross_query_str = "Please summarize the BioE80 course"
response = agent_chain.run("Please summarize the BioE80 course")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: Summarize the BioE80 course
Observation: 
BioE80 is an introductory course to bioengineering, which focuses on the systematic engineering of living systems to benefit all people and the planet. The course aims to help students learn ways of thinking about engineering living matter, empower them to explore and do bioengineering starting from DNA, and become more capable of learning and explaining bioengineering to themselves and others. Additionally, the course seeks to enable students to devise and express their wishes for bioengineering as might be made true by or before 2030, and to develop practical plans for making their wishes real.


> Finished chain.


In [12]:
cross_query_str = "Will Bioe80 be offered again next year?"
response = agent_chain.run(input=cross_query_str)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: Will Bioe80 be offered again next year?
Observation: 
It is not possible to answer this question with the given context information.


> Finished chain.


In [13]:
# demonstrate memory
agent_chain.run(input="What's my name?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: "What's my name?"
Observation: 
Your name is Ahmed Best.


> Finished chain.


'\nYour name is Ahmed Best.'

In [14]:
# reinitialize agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-4
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory, verbose=True)

In [15]:
# while True:
#     text_input = input("User: ")
#     response = agent_chain.run(input=text_input)
#     print(f"Agent: {response}")

In [17]:
from langchain.llms import OpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks import StreamlitCallbackHandler
import streamlit as st

memory = ConversationBufferMemory(memory_key="chat_history")
#llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-4

llm = OpenAI(temperature=0, streaming=True)
#tools = load_tools(["ddg-search"])
#already defined
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, memory=memory, verbose=True)

if prompt := st.chat_input():
    st.chat_message("user").write(prompt)
    with st.chat_message("assistant"):
        st_callback = StreamlitCallbackHandler(st.container())
        response = agent.run(prompt, callbacks=[st_callback])
        st.write(response)

2023-08-07 12:27:43.480 
  command:

    streamlit run /Users/jan/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
